# Midterm Project Messy Work
*Matthew Ueckermann*


## Importing the data

Can import one year as:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from  scipy import stats 
import time

df_2019 = pd.read_csv('https://enviro.epa.gov/enviro/efservice/MV_TRI_BASIC_DOWNLOAD/year/=/2019/fname/TRI_2019_US.csv/CSV',low_memory=False)
# to scrape later, do in for loop. Add time delay for politenes

print("done")

In [ ]:
df_2019.head()

In [ ]:
# Want to select/keep only the data that we care about:
#df_2019_trimmed = df_2019[["1. YEAR","2. TRIFD","8. ST","15. PARENT CO NAME","20. INDUSTRY SECTOR","34. CHEMICAL","38. CLEAN AIR ACT CHEMICAL","39. CLASSIFICATION","40. METAL","42. CARCINOGEN","45. 5.1 - FUGITIVE AIR","46. 5.2 - STACK AIR","47. 5.3 - WATER","51. 5.5.1 - LANDFILLS","59. ON-SITE RELEASE TOTAL","82. OFF-SITE RELEASE TOTAL","88. OFF-SITE RECYCLED TOTAL","101. TOTAL RELEASES","113. PRODUCTION WSTE (8.1-8.7)","114. 8.8 - ONE-TIME RELEASE","115. PROD_RATIO_OR_ ACTIVITY","116. 8.9 - PRODUCTION RATIO"]]

df_2019_trimmed = df_2019[["1. YEAR","2. TRIFD","8. ST","15. PARENT CO NAME","20. INDUSTRY SECTOR","34. CHEMICAL","39. CLASSIFICATION","42. CARCINOGEN","45. 5.1 - FUGITIVE AIR","59. ON-SITE RELEASE TOTAL","82. OFF-SITE RELEASE TOTAL","101. TOTAL RELEASES","116. 8.9 - PRODUCTION RATIO"]]
# this probably covers more than I'll actually use, but I think the rest of the columns are too much for me....

del df_2019

In [ ]:
df_2019_trimmed.head()

In [ ]:
## time to scrape!
df = df_2019_trimmed

for x in range(1987,2019):
    address = 'https://enviro.epa.gov/enviro/efservice/MV_TRI_BASIC_DOWNLOAD/year/=/'+str(x)+'/fname/TRI_'+str(x)+'_US.csv/CSV'
    df_new = pd.read_csv(address,low_memory=False)
    
    df_new_trimmed = df_new[["1. YEAR","2. TRIFD","8. ST","15. PARENT CO NAME","20. INDUSTRY SECTOR","34. CHEMICAL","39. CLASSIFICATION","42. CARCINOGEN","45. 5.1 - FUGITIVE AIR","46. 5.2 - STACK AIR","47. 5.3 - WATER","59. ON-SITE RELEASE TOTAL","82. OFF-SITE RELEASE TOTAL","101. TOTAL RELEASES","116. 8.9 - PRODUCTION RATIO"]]

    df = df.append(df_new_trimmed, ignore_index=True)
    del df_new
    time.sleep(2)

In [ ]:
df.tail(1) # seems like we have everything imported/scraped...

## Lets start understanding the data that we have

Want to see general structure of the TRI, see if anything interesting pops up

In [ ]:
df["34. CHEMICAL"].value_counts()

In [ ]:
df["15. PARENT CO NAME"].value_counts()

In [ ]:
df["8. ST"].value_counts()

In [ ]:
df["2. TRIFD"].value_counts()

In [ ]:
# Group by values more interesting though...
df.groupby(["34. CHEMICAL","1. YEAR"])["101. TOTAL RELEASES"].sum()

### Some very general trends

Interested in types of chemical emissions over time, both general and not

In [ ]:
# Lets just look at total chemical releases by year. See if there's anything there
import seaborn as sns
import matplotlib.pyplot as plt

sumEmm_year = df.groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum()

sumEmm_year.plot(kind='bar')
#not super exciting, might be able to attribute first decrease to the creation of the TRI, but probs not more

In [ ]:
# Continue, but lets look at specific bad chemcials, starting with carcinogens

sumCarcinogens_year = df[df["42. CARCINOGEN"]=="YES"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum()
sumCarcinogens_year.plot(kind='bar')

In [ ]:
# Dioxins

sumDiox_year = df[df["39. CLASSIFICATION"]=="Dioxin"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum()
sumDiox_year.plot(kind='bar')

In [ ]:
# PBTs

sumPBT_year = df[df["39. CLASSIFICATION"]=="PBT"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum()
sumPBT_year.plot(kind='bar')

In [ ]:
# So obviously, PBT/dioxin usage not decreasing... Is this due to addition of new compounds, or just greater use?

# Lets take some example PBTs

# lead
sumLead_year = df[df["34. CHEMICAL"]=="Lead"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum()
sumLead_year.plot(kind='bar')

# looks like they added more industries in 199 time, a well as more PBTs..

In [ ]:
df["20. INDUSTRY SECTOR"].value_counts()
#  Wow this is a lot. Really only had chemicals/petroleum in my mind

In [ ]:
#Lets look at the magnitude...
sumChem_industry = df[df["1. YEAR"]>2001].groupby(["20. INDUSTRY SECTOR"])["101. TOTAL RELEASES"].sum()
sumChem_industry.plot(kind="bar")

In [ ]:
sumChem_byIndustry_year = df.groupby(["1. YEAR","20. INDUSTRY SECTOR"])["101. TOTAL RELEASES"].sum()*10**-6
sumChem_byIndustry_year.unstack(level=1).plot.bar(stacked=True).legend(loc='center left',bbox_to_anchor=(1.0, 0.5));
plt.xlabel("Year")
plt.ylabel("Total chemicals emmited (Mlb)")
plt.show()

# Pretty chart, but think I will use to justify zeroing on just the chemicals industry...

### Zeroing in on the Chemical Industry

Seems like an interesting case study of the data. Want to look at the same trends as above and characterize

In [ ]:
df_chemical = df[df["20. INDUSTRY SECTOR"]=="Chemicals"]
df_chemical

In [ ]:
JM_df = df_chemical[df_chemical["15. PARENT CO NAME"]=="JOHNSON MATTHEY INC"] # my future employeer, might keep around if interested later...

In [ ]:
# All chemicals
df_chemical.groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum().plot(kind='bar') # hey you can do this in one line
### OOOOh a nice decrease with the introduction of the TRI, no longer confounded with the introduction of other industries

In [ ]:
# Carinogens
# start only looking at chemical...
sumDioxChem_year = df_chemical[df_chemical["42. CARCINOGEN"]=="YES"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum().plot(kind='bar') 

In [ ]:
# Dioxins
sumDioxChem_year = df_chemical[df_chemical["39. CLASSIFICATION"]=="Dioxin"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum().plot(kind='bar') # Methodology change to more group contribution in 2008
# counfounded by normalizations - take into account

In [ ]:
# PBT
sumDioxChem_year = df_chemical[df_chemical["39. CLASSIFICATION"]=="PBT"].groupby(["1. YEAR"])["101. TOTAL RELEASES"].sum().plot(kind='bar') # Methodology change to more group contribution in 2008
# confounded by additions - take into account

### Anaylsis ideas

Goals/ideas I had to look into at this point:
 - Look into controlling for production increase
 - Start to look at specific sites/companies
     - Case study notable chemicals (PFAS?)
 - Trends in emission types out of what I have now
 - Start making own stats
     - Yearly decrease in specific chemical emitted
         - compare with production increase ratio
     - Shift in chemicals compositions within plants
  - Look at state trends


## Do plants get better at controlling emissions?

Ended up not being able to get anything substantial to run here... spent a lot of time trying to control for production ratio and taking the difference between a emissions controlled by the production ratio one year compared to emissions in the previous year. Retrospectively, I think I wrote the loop wrong, but I also think I was asking way too much to look at emissions for each chemical in each facilitiy.

In [ ]:
year_location = df_chemical.groupby(["1. YEAR","2. TRIFD"])["101. TOTAL RELEASES"].sum()

year_location

#df_chemical.groupby(["1. YEAR","2. TRIFD"])["116. 8.9 - PRODUCTION RATIO"].sum()

#df_chemical.groupby(["2. TRIFD"])["101. TOTAL RELEASES"].count()

In [ ]:
df_chemical.groupby(["1. YEAR","2. TRIFD"])["101. TOTAL RELEASES"].describe()

In [ ]:
#Would want to control by increase in production, ratio is given in last column
#https://www.epa.gov/sites/production/files/2016-05/documents/p2_factsheet_may_2016_final.pdf how to think
df_chemical["116. 8.9 - PRODUCTION RATIO"]

df_chem_prod = df_chemical[df_chemical["116. 8.9 - PRODUCTION RATIO"]!=0] # Can now control by increase in production by facility

In [ ]:
specificSite_Chem = df_chem_prod[(df_chem_prod["2. TRIFD"]=="77015FRMNT2239H") & (df_chemical["34. CHEMICAL"]=="Hydrogen cyanide")]
#specificSite_Chem["Normalized Production"] - specificSite_Chem["101. TOTAL RELEASES"] Does not compare between years
specificSite_Chem # Can try to determine a statistic with this

In [ ]:
#specificSite_Chem.reset_index() dont think this is actually needed

In [ ]:
norm_year_prev = specificSite_Chem["101. TOTAL RELEASES"]/specificSite_Chem["116. 8.9 - PRODUCTION RATIO"]
specificSite_Chem["Normalized Production"] = norm_year_prev
specificSite_Chem["Normalized Difference"] = np.nan
specificSite_Chem


In [ ]:
specificSite_Chem.iat[5,11]
for i in range(2,32):
    specificSite_Chem.iat[i,16] = specificSite_Chem.iat[i,15]-specificSite_Chem.iat[i-1,11]

In [ ]:
specificSite_Chem # Negative normalized difference - actual lb difference accounting for increase in production. Positive indicates increased greater than expected due to production increase

In [ ]:
# Should have dropped the NaN, but I like the new Normalized difference metric. Can do anaylsis on it (mean, std...) to see how it shifts
specificSite_Chem["Normalized Difference"].describe()

In [ ]:
df_chem_prod.loc[specificSite_Chem.index,"Normalized Difference"] = specificSite_Chem["Normalized Difference"]  

## General process for site/chemical statistic:
Collect all years of one site and chemical
Divide total releases by production ratio, call normalized release
Subtract normalized release by actual release of previous year
Then have valid reduction indicator

For rest of dataset:

In [ ]:
# Code summary
# specificSite_Chem = df_chem_prod[(df_chem_prod["2. TRIFD"]=="77015FRMNT2239H") & (df_chemical["34. CHEMICAL"]=="Hydrogen cyanide")]
# norm_year_prev = specificSite_Chem["101. TOTAL RELEASES"]/specificSite_Chem["116. 8.9 - PRODUCTION RATIO"]
# specificSite_Chem["Normalized Production"] = norm_year_prev
# specificSite_Chem["Normalized Difference"] = np.nan
# for i in range(2,32):
#    specificSite_Chem.iat[i,18] = specificSite_Chem.iat[i,17]-specificSite_Chem.iat[i-1,13]
    # I just want to say this took me forever to think through

In [ ]:
df_chem_prod["Normalized Production"] = df_chem_prod["101. TOTAL RELEASES"]/df_chem_prod["116. 8.9 - PRODUCTION RATIO"] # want to normalize by first emmission?


In [ ]:
df_chem_prod["Normalized Difference"] = np.nan

In [ ]:
#This is probably pretty darn bad, but hey:

#for i in df_chem_prod["2. TRIFD"]:
#    for j in df_chem_prod[df_chem_prod["2. TRIFD"]==i]["34. CHEMICAL"]:
#        if(df_chem_prod[(df_chem_prod["2. TRIFD"]==i) & (df_chemical["34. CHEMICAL"]==i)].shape[0]==32):
#            specificSite_chem = df_chem_prod[(df_chem_prod["2. TRIFD"]==i) & (df_chemical["34. CHEMICAL"]==i)]
#            for i in range(2,32):
#                specificSite_Chem.iat[i,16] = specificSite_Chem.iat[i,15]-specificSite_Chem.iat[i-1,11]
#            df_chem_prod.loc[specificSite_Chem.index,"Normalized Difference"] = specificSite_Chem["Normalized Difference"]
#
# Gave over an hour to run and no dice... Oh well...

In [ ]:
df_chem_prod.dropna(subset=["Normalized Difference"]) # worked for the specific example...

### Emission Type

Where I go through emission data just for the chemical industry, think this is much more relevant and usable.

In [ ]:
fugAir = df_chemical.groupby(["1. YEAR"])["45. 5.1 - FUGITIVE AIR"].sum()
stackAir = df_chemical.groupby(["1. YEAR"])["46. 5.2 - STACK AIR"].sum()
waterAir = df_chemical.groupby(["1. YEAR"])["47. 5.3 - WATER"].sum()
df_emissionType = pd.concat({"Fugitive Air":fugAir,"Stack Air":stackAir,"Water":waterAir},axis=1)
df_emissionType.plot.bar(y=['Fugitive Air', "Stack Air"], stacked=True)

In [ ]:
df_emissionType.plot.bar(y=['Fugitive Air', "Stack Air","Water"], stacked=True)

In [ ]:
onsite = df_chemical.groupby(["1. YEAR"])["59. ON-SITE RELEASE TOTAL"].sum()*10**-6
offsite = df_chemical.groupby(["1. YEAR"])["82. OFF-SITE RELEASE TOTAL"].sum()*10**-6
df_emissionLocation = pd.concat({"Onsite":onsite,"Offsite":offsite},axis=1)

In [ ]:
df_emissionLocation.plot.bar(y=['Onsite', "Offsite"], stacked=True)
plt.xlabel("Year")
plt.ylabel("Total amount of chemicals emitted (Mlb)")
plt.title("Onsite to Offsite Emissions in the Chemical Industry")
plt.show()

### State Trends

Kinda want to create my own statistic/value to test/look for trends, lose ability to use the production ratio depending on how this is done

Looking first at yearly total releases again:

In [ ]:
year_state = df_chemical.groupby(["1. YEAR","8. ST"])["101. TOTAL RELEASES"].sum()*10**-6
year_state = year_state.reset_index()
year_state

In [ ]:
year_state = year_state.drop(index=year_state[year_state["8. ST"]=="VI"].index) # Not state
year_state = year_state.drop(index=year_state[year_state["8. ST"]=="PR"].index) # Not state
year_state = year_state.drop(index=year_state[year_state["8. ST"]=="DC"].index) # Not state, remove both as small
year_state = year_state.drop(year_state[year_state["8. ST"]=="AK"].index)       # Does not report after 2008
year_state = year_state.drop(year_state[year_state["8. ST"]=="HI"].index)       # Does not report after 2013

In [ ]:
year_state.groupby(["1. YEAR"])["101. TOTAL RELEASES"].describe()

In [ ]:
state_max = year_state.groupby(["8. ST"])["101. TOTAL RELEASES"].max()
state_min = year_state.groupby(["8. ST"])["101. TOTAL RELEASES"].min()
state_max

In [ ]:
state_min

In [ ]:
state_percent_decrease = (state_max-state_min)/state_max *100 # Percent decrease in TRI of largest emmission year in comparison to smallest
state_percent_decrease

In [ ]:
state_percent_decrease.describe()

In [ ]:
#From before I removed Alaska from the data set
# Alaska is pretty surprising, lets check why:
#year_state[year_state["8. ST"]=="AK"].plot.bar(x="1. YEAR",y="101. TOTAL RELEASES") # might be an artifact of the chemical industry itself pullin out of the state
# Stopped reporting in 2008. I guess remove from anaylsis

In [ ]:
#year_state.drop(year_state[year_state["8. ST"]=="AK"].index)

In [ ]:
#Can visualize set with a box and wiskers plot:
df_state_percent = pd.DataFrame(state_percent_decrease)
df_state_percent.columns = ['Percent decrease']
df_state_percent.drop(df_state_percent[])

In [ ]:
# Violin plots are cooler IMO
import seaborn as sns
sns.violinplot(y='Percent decrease', data = df_state_percent)
# the outliers confuse me tho

In [ ]:
sns.swarmplot(x='Percent decrease', data = df_state_percent)
plt.ylabel("Reduction from Highest Chemical Emission (%)")
plt.xlabel
# I like this the best

In [ ]:
sns.boxplot(x='Percent decrease', data = df_state_percent)

In [ ]:
df_state_percent.describe()

In [ ]:
df_state_percent.index.names = ["State"]
df_state_percent

In [ ]:
#Shame
df_state_percent.sort_values(by='Percent decrease', ascending = False)

In [ ]:
#Just realized that instead of min and max I should hardcode the years... Data is more confusing. Lets stick the other way and ankwoledge not ideal...
state_1987 = year_state[year_state["1. YEAR"]==1987]
state_2019 = year_state[year_state["1. YEAR"]==2019]

In [ ]:
# bad way to create series I want:
state_emmit_1987 = state_1987.groupby(["8. ST"])["101. TOTAL RELEASES"].max()
state_emmit_2019 = state_2019.groupby(["8. ST"])["101. TOTAL RELEASES"].min()

In [ ]:
state_emmit_1987

In [ ]:
state_percent_decrease = (state_emmit_1987-state_emmit_2019)/state_emmit_1987 *100

In [ ]:
state_percent_decrease

In [ ]:
# More sophisticated: look at each chemical released
chem_year_state = df_chemical.groupby(["1. YEAR","8. ST","34. CHEMICAL"])["101. TOTAL RELEASES"].sum()*10**-6
df_chem_state_year = chem_year_state.reset_index() # the multi-index hurts my head
df_chem_state_year
# too difficult to deal with for now

## Adding state ideology

Wanted to compare with some measure of a states ideology. Found a measure [here](https://rcfording.com/state-ideology-data/), which gives a nominate measure of state government ideology. The way that this is calculated is delatied [here](https://journals-sagepub-com.udel.idm.oclc.org/doi/10.1177/153244001001000201), but it essentially looks at the votes of a states congressional delegation as well as interest group ratings and extrapolates them to similar party state politicains. Its on a scale of 0-100 going from conservative-liberal.

Note I could not find an easy way to scape, and did some of my own pre-processing in Excel (adding abbreviations, deleting empty rows)

In [ ]:
# Dataset on state ideology

df_stIdeology = pd.read_csv("Data/stateideology_v2018.csv") # could not find out how to scrape
df_stIdeology

In [ ]:
# Want to create dataset of slopes in yearly emissions and correlate with ideology data
year_state["Normalized Release"] = np.nan
year_state["Difference in Release"]= np.nan
year_state["Ideology score"] = np.nan
year_state["Difference in Ideology score"] = np.nan

In [ ]:
year_state

In [ ]:
#year_state[year_state["8. ST"]=="AL"]

#year_state[(year_state["8. ST"]=="AL") & (year_state["1. YEAR"]==1998)]["Normalized Release"] = 1
#year_state.loc[((year_state["8. ST"]=="AL") & (year_state["1. YEAR"]==1998)),"Normalized Release"] = 1 # probably what I should have done before

#year_state[(year_state["8. ST"]=="AL") & (year_state["1. YEAR"]==1998)]["101. TOTAL RELEASES"]

In [ ]:
pd.options.mode.chained_assignment = None
for state in year_state[year_state["1. YEAR"]==1987]["8. ST"]:
    #print(state) sanity check
    for year in range(1987,2018):
        original_val = year_state[(year_state["8. ST"]==state) & (year_state["1. YEAR"]==1987)]["101. TOTAL RELEASES"].values[0]
        current_val = year_state[(year_state["8. ST"]==state) & (year_state["1. YEAR"]==year)]["101. TOTAL RELEASES"].values[0]
        norm_val = current_val/original_val
        year_state.loc[((year_state["8. ST"]==state) & (year_state["1. YEAR"]==year)),"Normalized Release"] = norm_val
        
        ideology_score = df_stIdeology[(df_stIdeology["year"]==year)&(df_stIdeology["State Abbreviation"]==state)]["State ideology"].values[0]
        year_state.loc[((year_state["8. ST"]==state) & (year_state["1. YEAR"]==year)),"Ideology score"] = ideology_score
        
        if(year!=1987):
            pre_val = year_state[(year_state["8. ST"]==state) & (year_state["1. YEAR"]==(year-1))]["Normalized Release"].values[0]
            difference = norm_val-pre_val
            year_state.loc[((year_state["8. ST"]==state) & (year_state["1. YEAR"]==year)),"Difference in Release"] = difference
            
            pre_ideology = year_state[(year_state["8. ST"]==state) & (year_state["1. YEAR"]==(year-1))]["Ideology score"].values[0]
            id_diff = ideology_score-pre_ideology
            
            year_state.loc[((year_state["8. ST"]==state) & (year_state["1. YEAR"]==year)),"Difference in Ideology score"] = id_diff

In [ ]:
year_state = year_state.drop(year_state[year_state["1. YEAR"]==1987].index) #remove baseline - removes straightlines in dataset
year_state = year_state.dropna()

In [ ]:

sns.scatterplot(y = "Normalized Release", x = "Ideology score", data = year_state)
plt.ylim(0,2) # some outliers exist above

In [ ]:
# Confirm with Pearson R:

[r,p] = stats.pearsonr(year_state["Ideology score"],year_state["Normalized Release"])
print(r)
# huh I guess a significant, albeit low r 

In [ ]:
# Know, want to determine if there is any correlation between the yearly difference in release and ideology:
# Lets start with plotting:
sns.scatterplot(y = "Difference in Release", x = "Difference in Ideology score", data = year_state)
plt.ylim(-2, 2)
plt.xlabel("Change in Ideology Score")
plt.ylabel("Change in Normalized Emissions")
# Dont see any trend

In [ ]:
stats.pearsonr(year_state["Difference in Ideology score"],year_state["Difference in Release"])

In [ ]:
sns.scatterplot(y = "Normalized Release", x = "1. YEAR", data = year_state)
plt.ylim(0,2)
plt.ylabel("Difference in Normalized Release")

In [ ]:
year_state[year_state["1. YEAR"]==1987]["8. ST"]

In [ ]:
## Think I am done, would have liked to do something with plant data or specific chemical data, but this has been a lot